# Preprocessing Dataset
---

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
## Importing required libraries
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings

from time import time
from sklearn.model_selection import (
    train_test_split,
    StratifiedShuffleSplit,
    GridSearchCV
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
#from sklearn import model_selection

from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline

In [3]:
sns.set_style('whitegrid')

warnings.simplefilter("ignore")

In [4]:
# https://www.kaggle.com/hugomathien/soccer

conn = sqlite3.connect('./soccer/database.sqlite')

In [5]:
player_df = pd.read_sql_query("SELECT * FROM Player", conn)
player_stats_df = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)
team_df = pd.read_sql_query("SELECT * FROM Team", conn)
match_df = pd.read_sql_query("SELECT * FROM Match", conn)

In [6]:
print("Shape 'player_df' = {}".format(player_df.shape))
display(player_df.head(10))
print('=' * 30)
display(player_df.dtypes)

Shape 'player_df' = (11060, 7)


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
5,6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161
6,7,564793,Aaron Kuhl,221280,1996-01-30 00:00:00,172.72,146
7,8,30895,Aaron Lennon,152747,1987-04-16 00:00:00,165.10,139
8,9,528212,Aaron Lennox,206592,1993-02-19 00:00:00,190.50,181
9,10,101042,Aaron Meijers,188621,1987-10-28 00:00:00,175.26,170


id                      int64
player_api_id           int64
player_name            object
player_fifa_api_id      int64
birthday               object
height                float64
weight                  int64
dtype: object

In [7]:
print("Shape 'player_stats_df' = {}".format(player_stats_df.shape))
display(player_stats_df.head(10))
print('=' * 30)
display(player_stats_df.dtypes)

Shape 'player_stats_df' = (183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
7,8,189615,155782,2016-01-07 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
8,9,189615,155782,2015-12-24 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
9,10,189615,155782,2015-12-17 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


id                       int64
player_fifa_api_id       int64
player_api_id            int64
date                    object
overall_rating         float64
potential              float64
preferred_foot          object
attacking_work_rate     object
defensive_work_rate     object
crossing               float64
finishing              float64
heading_accuracy       float64
short_passing          float64
volleys                float64
dribbling              float64
curve                  float64
free_kick_accuracy     float64
long_passing           float64
ball_control           float64
acceleration           float64
sprint_speed           float64
agility                float64
reactions              float64
balance                float64
shot_power             float64
jumping                float64
stamina                float64
strength               float64
long_shots             float64
aggression             float64
interceptions          float64
positioning            float64
vision  

In [8]:
player_stats_df[player_stats_df['player_api_id'] == 505942]

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [9]:
print("Shape 'team_df' = {}".format(team_df.shape))
display(team_df.head(10))
print('=' * 30)
display(team_df.dtypes)

Shape 'team_df' = (299, 5)


,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
5,6,8635,229.0,RSC Anderlecht,AND
6,7,9991,674.0,KAA Gent,GEN
7,8,9998,1747.0,RAEC Mons,MON
8,9,7947,NaN,FCV Dender EH,DEN
9,10,9985,232.0,Standard de Liège,STL


id                    int64
team_api_id           int64
team_fifa_api_id    float64
team_long_name       object
team_short_name      object
dtype: object

In [10]:
#match_df.isna().sum()

In [11]:
print("Shape 'match_df_OLD' = {}".format(match_df.shape))

# Reduce match data to fulfill run time requirements
columns = [
    "country_id", "league_id", "season", "stage", "date", "match_api_id",
    "home_team_api_id",  "away_team_api_id", "home_team_goal", "away_team_goal",
    "home_player_1", "home_player_2", "home_player_3", "home_player_4", "home_player_5",
    "home_player_6", "home_player_7",  "home_player_8", "home_player_9", "home_player_10",
    "home_player_11",
    "away_player_1", "away_player_2", "away_player_3", "away_player_4", "away_player_5",
    "away_player_6", "away_player_7", "away_player_8", "away_player_9", "away_player_10",
    "away_player_11"
]
match_df.dropna(subset=columns, inplace=True)
match_df = match_df.tail(1500)

for i in range(1, 12):
    match_df['home_player_{}'.format(i)] = match_df['home_player_{}'.format(i)].astype(int)
    match_df['away_player_{}'.format(i)] = match_df['away_player_{}'.format(i)].astype(int)

Shape 'match_df_OLD' = (25979, 115)


In [12]:
print("Shape 'match_df' = {}".format(match_df.shape))
display(match_df.head(10))
print('=' * 30)
display(match_df.dtypes)

Shape 'match_df' = (1500, 115)


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
24207,24208,21518,21518,2015/2016,12,2015-11-23 00:00:00,2030192,8305,8370,1,...,NaN,2.50,3.40,3.00,NaN,NaN,NaN,NaN,NaN,NaN
24208,24209,21518,21518,2015/2016,12,2015-11-22 00:00:00,2030193,10205,8372,1,...,NaN,1.53,4.20,7.50,NaN,NaN,NaN,NaN,NaN,NaN
24209,24210,21518,21518,2015/2016,12,2015-11-21 00:00:00,2030194,8633,8634,0,...,NaN,2.45,3.75,2.80,NaN,NaN,NaN,NaN,NaN,NaN
24210,24211,21518,21518,2015/2016,12,2015-11-21 00:00:00,2030196,8560,8302,2,...,NaN,2.70,3.60,2.70,NaN,NaN,NaN,NaN,NaN,NaN
24211,24212,21518,21518,2015/2016,12,2015-11-21 00:00:00,2030197,10267,8306,1,...,NaN,1.53,4.33,7.00,NaN,NaN,NaN,NaN,NaN,NaN
24212,24213,21518,21518,2015/2016,12,2015-11-21 00:00:00,2030198,9783,9910,2,...,NaN,3.30,3.60,2.25,NaN,NaN,NaN,NaN,NaN,NaN
24213,24214,21518,21518,2015/2016,12,2015-11-22 00:00:00,2030199,9869,8581,0,...,NaN,2.20,3.30,3.75,NaN,NaN,NaN,NaN,NaN,NaN
24214,24215,21518,21518,2015/2016,12,2015-11-22 00:00:00,2030200,8603,9906,0,...,NaN,6.25,3.90,1.65,NaN,NaN,NaN,NaN,NaN,NaN
24215,24216,21518,21518,2015/2016,12,2015-11-21 00:00:00,2030201,8558,9864,2,...,NaN,2.40,3.25,3.30,NaN,NaN,NaN,NaN,NaN,NaN
24216,24217,21518,21518,2015/2016,12,2015-11-22 00:00:00,2030202,7878,8315,2,...,NaN,3.80,3.50,2.10,NaN,NaN,NaN,NaN,NaN,NaN


id                    int64
country_id            int64
league_id             int64
season               object
stage                 int64
date                 object
match_api_id          int64
home_team_api_id      int64
away_team_api_id      int64
home_team_goal        int64
away_team_goal        int64
home_player_X1      float64
home_player_X2      float64
home_player_X3      float64
home_player_X4      float64
home_player_X5      float64
home_player_X6      float64
home_player_X7      float64
home_player_X8      float64
home_player_X9      float64
home_player_X10     float64
home_player_X11     float64
away_player_X1      float64
away_player_X2      float64
away_player_X3      float64
away_player_X4      float64
away_player_X5      float64
away_player_X6      float64
away_player_X7      float64
away_player_X8      float64
                     ...   
B365H               float64
B365D               float64
B365A               float64
BWH                 float64
BWD                 

In [13]:
def get_fifa_data(matches, player_stats):
    """
    Obtenemos las estadisticas mas recientes
    para todos los partidos.
    """ 
    # Apply get_fifa_stats for each match (each row)
    fifa_data = matches.apply(lambda x: get_fifa_stats(x, player_stats), axis=1)

    return fifa_data


def get_fifa_stats(match, player_stats):
    """
    Obtenemos las estadisticas mas recientes de los jugadores
    dado un partido.
    """
    #Define variables
    match_id =  match.match_api_id
    date = match.date
    players = [
        "home_player_1","home_player_2", "home_player_3", "home_player_4",
        "home_player_5", "home_player_6", "home_player_7", "home_player_8",
        "home_player_9", "home_player_10", "home_player_11",
        "away_player_1", "away_player_2", "away_player_3", "away_player_4",
        "away_player_5", "away_player_6", "away_player_7", "away_player_8",
        "away_player_9", "away_player_10", "away_player_11"
    ]

    player_stats_new = pd.DataFrame()
    names = []
    
    for player in players:           
        player_id = match[player]
        
        stats = player_stats[player_stats.player_api_id == player_id]
            
        # Estadisticas mas recientes
        current_stats = stats[stats.date < date]
        current_stats = current_stats.sort_values(by='date', ascending=False)
        current_stats = current_stats[:1]
        
        if np.isnan(player_id) == True:
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace=True, drop=True)
            overall_rating = pd.Series(current_stats.loc[0, "overall_rating"])

        name = "{}_overall_rating".format(player)
        names.append(name)
            
        # Aggregate stats
        player_stats_new = pd.concat([player_stats_new, overall_rating], axis=1)

    player_stats_new.columns = names
    player_stats_new['match_api_id'] = match_id
    player_stats_new.reset_index(inplace=True, drop=True)

    return player_stats_new.iloc[0]

In [14]:
%time

fifa_data = get_fifa_data(match_df, player_stats_df)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [15]:
print("Shape 'fifa_data' = {}".format(fifa_data.shape))
display(fifa_data.head())
print('=' * 30)
display(fifa_data.dtypes)

Shape 'fifa_data' = (1500, 23)


,home_player_1_overall_rating,home_player_2_overall_rating,home_player_3_overall_rating,home_player_4_overall_rating,home_player_5_overall_rating,home_player_6_overall_rating,home_player_7_overall_rating,home_player_8_overall_rating,home_player_9_overall_rating,home_player_10_overall_rating,...,away_player_3_overall_rating,away_player_4_overall_rating,away_player_5_overall_rating,away_player_6_overall_rating,away_player_7_overall_rating,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,match_api_id
24207,77.0,69.0,74.0,73.0,74.0,74.0,76.0,77.0,76.0,74.0,...,70.0,75.0,75.0,80.0,78.0,72.0,71.0,73.0,73.0,2030192.0
24208,76.0,79.0,73.0,80.0,77.0,77.0,79.0,81.0,78.0,77.0,...,73.0,70.0,75.0,72.0,71.0,74.0,76.0,71.0,72.0,2030193.0
24209,81.0,81.0,82.0,87.0,83.0,87.0,87.0,87.0,87.0,86.0,...,85.0,83.0,84.0,84.0,86.0,88.0,78.0,90.0,88.0,2030194.0
24210,79.0,65.0,76.0,80.0,71.0,79.0,77.0,76.0,77.0,79.0,...,79.0,77.0,81.0,79.0,82.0,79.0,81.0,81.0,79.0,2030196.0
24211,74.0,73.0,82.0,81.0,80.0,82.0,80.0,80.0,81.0,80.0,...,72.0,71.0,69.0,69.0,71.0,69.0,67.0,76.0,76.0,2030197.0


home_player_1_overall_rating     float64
home_player_2_overall_rating     float64
home_player_3_overall_rating     float64
home_player_4_overall_rating     float64
home_player_5_overall_rating     float64
home_player_6_overall_rating     float64
home_player_7_overall_rating     float64
home_player_8_overall_rating     float64
home_player_9_overall_rating     float64
home_player_10_overall_rating    float64
home_player_11_overall_rating    float64
away_player_1_overall_rating     float64
away_player_2_overall_rating     float64
away_player_3_overall_rating     float64
away_player_4_overall_rating     float64
away_player_5_overall_rating     float64
away_player_6_overall_rating     float64
away_player_7_overall_rating     float64
away_player_8_overall_rating     float64
away_player_9_overall_rating     float64
away_player_10_overall_rating    float64
away_player_11_overall_rating    float64
match_api_id                     float64
dtype: object

In [16]:
def get_last_matches(matches, date, team, x=10):
    """
    Obtiene los ultimos 'x' partidos de un equipo.
    """
    team_matches = matches[(matches.home_team_api_id == team) | (matches.away_team_api_id == team)]

    last_matches = team_matches[team_matches.date < date]
    last_matches = last_matches.sort_values(by='date', ascending=False)
    last_matches = last_matches.iloc[0:x, :]

    return last_matches


def get_last_matches_against_eachother(matches, date, home_team, away_team, x=10):
    """
    Obtiene los ultimos 'x' partidos entre dos equipos.
    """
    # Find matches of both teams
    home_matches = matches[(matches.home_team_api_id == home_team) & (matches.away_team_api_id == away_team)]
    away_matches = matches[(matches.home_team_api_id == away_team) & (matches.away_team_api_id == home_team)]
    total_matches = pd.concat([home_matches, away_matches])

    last_matches = total_matches[total_matches.date < date]
    last_matches = last_matches.sort_values(by='date', ascending=False)
    try:
        last_matches = last_matches.iloc[0:x, :]
    except Exception:
        last_matches = last_matches.iloc[0:total_matches.shape[0], :]
        
        # Check for error in data
        if last_matches.shape[0] > x:
            print("Error in obtaining matches")
            
    return last_matches


def get_goals(matches, team, option="GF"):
    """
    Obtenemos la totalidad de goles a favor o goels en contra
    de un equipo en un conjunto de partidos.
    """
    # GF: Goals For
    # GA: Goals Against

    if option == "GF":
        home_goals = int(matches.home_team_goal[matches.home_team_api_id == team].sum())
        away_goals = int(matches.away_team_goal[matches.away_team_api_id == team].sum())
    elif option == "GA":
        home_goals = int(matches.home_team_goal[matches.away_team_api_id == team].sum())
        away_goals = int(matches.away_team_goal[matches.home_team_api_id == team].sum())

    total_goals = home_goals + away_goals

    return total_goals


def get_wins(matches, team):
    """
    Obtenemos la cantidad de victoriasde un equipo
    en un conjunto de partidos.
    """
    home_wins = int(matches.home_team_goal[(matches.home_team_api_id == team) & (matches.home_team_goal > matches.away_team_goal)].count())
    away_wins = int(matches.away_team_goal[(matches.away_team_api_id == team) & (matches.away_team_goal > matches.home_team_goal)].count())

    total_wins = home_wins + away_wins

    return total_wins  

In [17]:
def get_match_features(match, matches, x=10):
    """
    Create match specific features for a given match.
    """
    date = match['date']
    home_team = match['home_team_api_id']
    away_team = match['away_team_api_id']
    
    matches_home_team = get_last_matches(matches, date, home_team, x=10)  # Ultimos 10 partidos del equipo local
    matches_away_team = get_last_matches(matches, date, away_team, x=10)  # Ultimos 10 partidos del equipo visitante

    # Ultimos 3 partidos entre los equipos locales y visitante
    last_matches_against = get_last_matches_against_eachother(matches, date, home_team, away_team, x=3)

    home_goals_for     = get_goals(matches_home_team, home_team, option="GF")  # Cantidad de goles a favor de los equipos locales
    home_goals_against = get_goals(matches_home_team, home_team, option="GA")  # Cantidad de goles en contra de los equipos locales
    away_goals_for     = get_goals(matches_away_team, away_team, option="GF")  # Cantidad de goles a favor de los equipos visitantes
    away_goals_against = get_goals(matches_away_team, away_team, option="GA")  # Cantidad de goles en contra de los equipos visitantes
    
    # Define result data frame
    result = pd.DataFrame()

    result.loc[0, 'match_api_id'] = match.match_api_id
    result.loc[0, 'league_id'] = match.league_id

    # Create match features
    result.loc[0, 'home_team_goals_difference'] = home_goals_for - home_goals_against
    result.loc[0, 'away_team_goals_difference'] = away_goals_for - away_goals_against
    result.loc[0, 'games_won_home_team']        = get_wins(matches_home_team, home_team) 
    result.loc[0, 'games_won_away_team']        = get_wins(matches_away_team, away_team)
    result.loc[0, 'games_against_won']          = get_wins(last_matches_against, home_team)
    result.loc[0, 'games_against_lost']         = get_wins(last_matches_against, away_team)
    
    # Return match features
    return result.loc[0]

In [18]:
%time

# Generamos nuevas features para todos los partidos
match_stats = match_df.apply(lambda x: get_match_features(x, match_df, x=10), axis=1)
match_stats = match_stats.astype(int)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [19]:
print("Shape 'match_stats' = {}".format(match_stats.shape))
display(match_stats.head())
print('=' * 30)
display(match_stats.dtypes)

Shape 'match_stats' = (1500, 8)


,match_api_id,league_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost
24207,2030192,21518,-5,-8,2,1,0,0
24208,2030193,21518,4,-2,4,1,0,0
24209,2030194,21518,18,7,6,6,0,0
24210,2030196,21518,1,1,2,3,0,0
24211,2030197,21518,1,1,3,1,0,0


match_api_id                  int64
league_id                     int64
home_team_goals_difference    int64
away_team_goals_difference    int64
games_won_home_team           int64
games_won_away_team           int64
games_against_won             int64
games_against_lost            int64
dtype: object

In [20]:
# Create dummies for league ID feature
dummies = pd.get_dummies(match_stats['league_id']).rename(columns = lambda x: 'League_{}'.format(str(x)))

In [21]:
print("Shape 'dummies' = {}".format(dummies.shape))
display(dummies.head())
print('=' * 30)
display(dummies.dtypes)

Shape 'dummies' = (1500, 2)


,League_21518,League_24558
24207,1,0
24208,1,0
24209,1,0
24210,1,0
24211,1,0


League_21518    uint8
League_24558    uint8
dtype: object

In [22]:
match_stats = pd.concat([match_stats, dummies], axis=1)
match_stats.drop(['league_id'], axis=1, inplace=True)

In [23]:
print("Shape 'match_stats' = {}".format(match_stats.shape))
display(match_stats.head())
print('=' * 30)
display(match_stats.dtypes)

Shape 'match_stats' = (1500, 9)


,match_api_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,League_21518,League_24558
24207,2030192,-5,-8,2,1,0,0,1,0
24208,2030193,4,-2,4,1,0,0,1,0
24209,2030194,18,7,6,6,0,0,1,0
24210,2030196,1,1,2,3,0,0,1,0
24211,2030197,1,1,3,1,0,0,1,0


match_api_id                  int64
home_team_goals_difference    int64
away_team_goals_difference    int64
games_won_home_team           int64
games_won_away_team           int64
games_against_won             int64
games_against_lost            int64
League_21518                  uint8
League_24558                  uint8
dtype: object

In [24]:
def get_match_label(match):
    """
    Agrega una nueva columna: Win, Draw o Lose

    Si el equipo local le gano al visitante    ===> Win
    Si el equipo local empato con el visitante ===> Draw
    Si el equipo local perdio con el visitante ===> Lose
    """
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
     
    label = pd.DataFrame()
    label.loc[0, 'match_api_id'] = match['match_api_id'] 

    if home_goals > away_goals:
        label.loc[0,'label'] = "Win"
    if home_goals == away_goals:
        label.loc[0,'label'] = "Draw"
    if home_goals < away_goals:
        label.loc[0,'label'] = "Lose"
  
    return label.loc[0]

In [25]:
%time

# Agregamos un nuevo label, segun el estado del partido
labels = match_df.apply(get_match_label, axis=1)
labels['match_api_id'] = labels['match_api_id'].astype(int)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [26]:
print("Shape 'labels' = {}".format(labels.shape))
display(labels.head())
print('=' * 30)
display(labels.dtypes)

Shape 'labels' = (1500, 2)


,match_api_id,label
24207,2030192,Draw
24208,2030193,Draw
24209,2030194,Lose
24210,2030196,Win
24211,2030197,Draw


match_api_id     int64
label           object
dtype: object

In [27]:
def get_bookkeeper_data(matches, bookkeepers):
    """
    Agrega datos contables para todos los partidos y
    para cada estado del partido.
    """
    bk_data = pd.DataFrame()

    for bookkeeper in bookkeepers:
        temp_data = matches.loc[:, matches.columns.str.contains(bookkeeper)]
        temp_data.loc[:, 'bookkeeper'] = str(bookkeeper)
        temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']

        cols = temp_data.columns.values
        cols[:3] = ['Win', 'Draw', 'Lose']
        temp_data.columns = cols
        temp_data.loc[:, 'Win'] = pd.to_numeric(temp_data['Win'])
        temp_data.loc[:, 'Draw'] = pd.to_numeric(temp_data['Draw'])
        temp_data.loc[:, 'Lose'] = pd.to_numeric(temp_data['Lose'])

        temp_data = convert_odds_to_prob(temp_data)
        temp_data.drop('match_api_id', axis=1, inplace=True)
        temp_data.drop('bookkeeper', axis=1, inplace=True)
        temp_data.columns.values[:3] = [
            "{}_Win".format(bookkeeper),
            "{}_Draw".format(bookkeeper),
            "{}_Lose".format(bookkeeper),
        ]

        # Aggregate data
        bk_data = pd.concat([bk_data, temp_data], axis=1)

    # Add match api id to data
    temp_data.loc[:, 'match_api_id'] = matches.loc[:, 'match_api_id']
    
    # Return bookkeeper data
    return bk_data


def convert_odds_to_prob(match_odds):
    """
    Convierte las probabilidades de contable en probabilidades.
    """
    match_id = match_odds.loc[:, 'match_api_id']
    bookkeeper = match_odds.loc[:, 'bookkeeper']    
    win_odd = match_odds.loc[:, 'Win']
    draw_odd = match_odds.loc[:, 'Draw']
    lose_odd = match_odds.loc[:, 'Lose']
    
    win_prob = 1 / win_odd
    draw_prob = 1 / draw_odd
    lose_prob = 1 / lose_odd
    total_prob = win_prob + draw_prob + lose_prob
    
    probs = pd.DataFrame()
    
    # Define output format and scale probs by sum over all probs
    probs.loc[:, 'match_api_id'] = match_id
    probs.loc[:, 'bookkeeper'] = bookkeeper
    probs.loc[:, 'Win'] = win_prob / total_prob
    probs.loc[:, 'Draw'] = draw_prob / total_prob
    probs.loc[:, 'Lose'] = lose_prob / total_prob
    
    return probs

In [28]:
%time

# Obtenemos la contabilidad para todos los partidos
bk_data = get_bookkeeper_data(match_df, bookkeepers=['B365', 'BW'])
bk_data.loc[:, 'match_api_id'] = match_df.loc[:, 'match_api_id']

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [29]:
print("Shape 'bk_data' = {}".format(bk_data.shape))
display(bk_data.head())
print('=' * 30)
display(bk_data.dtypes)

Shape 'bk_data' = (1500, 7)


,B365_Win,B365_Draw,B365_Lose,BW_Win,BW_Draw,BW_Lose,match_api_id
24207,0.395683,0.287770,0.316547,0.389704,0.287782,0.322514,2030192
24208,0.624582,0.238903,0.136516,0.612713,0.253365,0.133922,2030193
24209,0.390698,0.260465,0.348837,0.380496,0.279776,0.339728,2030194
24210,0.360875,0.287606,0.351519,0.354839,0.290323,0.354839,2030196
24211,0.651526,0.223380,0.125093,0.634855,0.224066,0.141079,2030197


B365_Win        float64
B365_Draw       float64
B365_Lose       float64
BW_Win          float64
BW_Draw         float64
BW_Lose         float64
match_api_id      int64
dtype: object

In [30]:
%time

# Unimos todas las features generadas en un solo DataFrame
features = pd.merge(match_stats, fifa_data, on='match_api_id', how='left')
features = pd.merge(features, bk_data, on='match_api_id', how='left')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [31]:
print("Shape 'features' = {}".format(features.shape))
display(features.head())
print('=' * 30)
display(features.dtypes)

Shape 'features' = (1500, 37)


,match_api_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,League_21518,League_24558,home_player_1_overall_rating,...,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,B365_Win,B365_Draw,B365_Lose,BW_Win,BW_Draw,BW_Lose
0,2030192,-5,-8,2,1,0,0,1,0,77.0,...,72.0,71.0,73.0,73.0,0.395683,0.287770,0.316547,0.389704,0.287782,0.322514
1,2030193,4,-2,4,1,0,0,1,0,76.0,...,74.0,76.0,71.0,72.0,0.624582,0.238903,0.136516,0.612713,0.253365,0.133922
2,2030194,18,7,6,6,0,0,1,0,81.0,...,88.0,78.0,90.0,88.0,0.390698,0.260465,0.348837,0.380496,0.279776,0.339728
3,2030196,1,1,2,3,0,0,1,0,79.0,...,79.0,81.0,81.0,79.0,0.360875,0.287606,0.351519,0.354839,0.290323,0.354839
4,2030197,1,1,3,1,0,0,1,0,74.0,...,69.0,67.0,76.0,76.0,0.651526,0.223380,0.125093,0.634855,0.224066,0.141079


match_api_id                       int64
home_team_goals_difference         int64
away_team_goals_difference         int64
games_won_home_team                int64
games_won_away_team                int64
games_against_won                  int64
games_against_lost                 int64
League_21518                       uint8
League_24558                       uint8
home_player_1_overall_rating     float64
home_player_2_overall_rating     float64
home_player_3_overall_rating     float64
home_player_4_overall_rating     float64
home_player_5_overall_rating     float64
home_player_6_overall_rating     float64
home_player_7_overall_rating     float64
home_player_8_overall_rating     float64
home_player_9_overall_rating     float64
home_player_10_overall_rating    float64
home_player_11_overall_rating    float64
away_player_1_overall_rating     float64
away_player_2_overall_rating     float64
away_player_3_overall_rating     float64
away_player_4_overall_rating     float64
away_player_5_ov

In [32]:
feables = pd.merge(features, labels, on='match_api_id', how='left')
feables.dropna(inplace=True)

In [33]:
print("Shape 'feables' = {}".format(feables.shape))
display(feables.head())
print('=' * 30)
display(feables.dtypes)

Shape 'feables' = (322, 38)


,match_api_id,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,League_21518,League_24558,home_player_1_overall_rating,...,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,B365_Win,B365_Draw,B365_Lose,BW_Win,BW_Draw,BW_Lose,label
0,2030192,-5,-8,2,1,0,0,1,0,77.0,...,71.0,73.0,73.0,0.395683,0.287770,0.316547,0.389704,0.287782,0.322514,Draw
1,2030193,4,-2,4,1,0,0,1,0,76.0,...,76.0,71.0,72.0,0.624582,0.238903,0.136516,0.612713,0.253365,0.133922,Draw
2,2030194,18,7,6,6,0,0,1,0,81.0,...,78.0,90.0,88.0,0.390698,0.260465,0.348837,0.380496,0.279776,0.339728,Lose
3,2030196,1,1,2,3,0,0,1,0,79.0,...,81.0,81.0,79.0,0.360875,0.287606,0.351519,0.354839,0.290323,0.354839,Win
4,2030197,1,1,3,1,0,0,1,0,74.0,...,67.0,76.0,76.0,0.651526,0.223380,0.125093,0.634855,0.224066,0.141079,Draw


match_api_id                       int64
home_team_goals_difference         int64
away_team_goals_difference         int64
games_won_home_team                int64
games_won_away_team                int64
games_against_won                  int64
games_against_lost                 int64
League_21518                       uint8
League_24558                       uint8
home_player_1_overall_rating     float64
home_player_2_overall_rating     float64
home_player_3_overall_rating     float64
home_player_4_overall_rating     float64
home_player_5_overall_rating     float64
home_player_6_overall_rating     float64
home_player_7_overall_rating     float64
home_player_8_overall_rating     float64
home_player_9_overall_rating     float64
home_player_10_overall_rating    float64
home_player_11_overall_rating    float64
away_player_1_overall_rating     float64
away_player_2_overall_rating     float64
away_player_3_overall_rating     float64
away_player_4_overall_rating     float64
away_player_5_ov

In [34]:
inputs = feables.drop('match_api_id', axis=1)

In [35]:
print("Shape 'inputs' = {}".format(inputs.shape))
display(inputs.head())
print('=' * 30)
display(inputs.dtypes)

Shape 'inputs' = (322, 37)


,home_team_goals_difference,away_team_goals_difference,games_won_home_team,games_won_away_team,games_against_won,games_against_lost,League_21518,League_24558,home_player_1_overall_rating,home_player_2_overall_rating,...,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,B365_Win,B365_Draw,B365_Lose,BW_Win,BW_Draw,BW_Lose,label
0,-5,-8,2,1,0,0,1,0,77.0,69.0,...,71.0,73.0,73.0,0.395683,0.287770,0.316547,0.389704,0.287782,0.322514,Draw
1,4,-2,4,1,0,0,1,0,76.0,79.0,...,76.0,71.0,72.0,0.624582,0.238903,0.136516,0.612713,0.253365,0.133922,Draw
2,18,7,6,6,0,0,1,0,81.0,81.0,...,78.0,90.0,88.0,0.390698,0.260465,0.348837,0.380496,0.279776,0.339728,Lose
3,1,1,2,3,0,0,1,0,79.0,65.0,...,81.0,81.0,79.0,0.360875,0.287606,0.351519,0.354839,0.290323,0.354839,Win
4,1,1,3,1,0,0,1,0,74.0,73.0,...,67.0,76.0,76.0,0.651526,0.223380,0.125093,0.634855,0.224066,0.141079,Draw


home_team_goals_difference         int64
away_team_goals_difference         int64
games_won_home_team                int64
games_won_away_team                int64
games_against_won                  int64
games_against_lost                 int64
League_21518                       uint8
League_24558                       uint8
home_player_1_overall_rating     float64
home_player_2_overall_rating     float64
home_player_3_overall_rating     float64
home_player_4_overall_rating     float64
home_player_5_overall_rating     float64
home_player_6_overall_rating     float64
home_player_7_overall_rating     float64
home_player_8_overall_rating     float64
home_player_9_overall_rating     float64
home_player_10_overall_rating    float64
home_player_11_overall_rating    float64
away_player_1_overall_rating     float64
away_player_2_overall_rating     float64
away_player_3_overall_rating     float64
away_player_4_overall_rating     float64
away_player_5_overall_rating     float64
away_player_6_ov

In [37]:
inputs.to_csv(
    './football_data.csv',
    index=False
)